https://huggingface.co/jxm/cde-small-v1


In [1]:
import transformers
import polars as pl
import os

tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")

/Users/maxwoolf/Library/Mobile Documents/com~apple~CloudDocs/PythonProjects/imdb-embeddings/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
device = "mps"

model = transformers.AutoModel.from_pretrained(
    "jxm/cde-small-v1", trust_remote_code=True
)

_ = model.to(device)

/Users/maxwoolf/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/c1b1fd7a715b8eb2e232d34593154ac782c98ac9/modeling_hf_nomic_bert.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related

Disabled 37 dropout modules from model type <class 'transformers_modules.jxm.cde-small-v1.10f081914816933d06b18c414c3484c96de49cdb.model.BiEncoder'>
modified 12 rotary modules – set rotary_start_pos to 512
Disabled 74 dropout modules from model type <class 'transformers_modules.jxm.cde-small-v1.10f081914816933d06b18c414c3484c96de49cdb.model.DatasetTransformer'>


In [3]:
model.device

device(type='mps', index=0)

In [4]:
data_dir = "/Users/maxwoolf/Downloads"

df = pl.read_parquet(os.path.join(data_dir, "test_movie_json_input.parquet"))
df = df.sample(fraction=1.0, shuffle=True, seed=42)
df

tconst,startYear,averageRating,json
str,i64,f64,str
"""tt0136535""",1998,4.8,"""{'title': 'Webmaster', 'genres…"
"""tt1933566""",1968,4.6,"""{'title': 'The daughter of the…"
"""tt2294804""",2008,7.3,"""{'title': 'Found Footage Festi…"
"""tt5297478""",2015,7.5,"""{'title': 'Boudewijn de Groot …"
"""tt0072358""",1974,6.5,"""{'title': 'Last Moments', 'gen…"
…,…,…,…
"""tt1241016""",2008,8.0,"""{'title': 'Family', 'genres': …"
"""tt19864536""",2022,6.9,"""{'title': 'Afrika', 'genres': …"
"""tt0810444""",2006,3.3,"""{'title': 'The Pumpkin Karver'…"


## First Stage


In [5]:
import torch
from tqdm.autonotebook import tqdm
import random

In [7]:
df_subset = df.sample(n=1600, shuffle=True, seed=42)
json_docs = df_subset["json"].to_list()

json_docs[:5]

["{'title': 'Mechtat i zhit', 'genres': ['Drama'], 'is_adult': False, 'release_year': 1975, 'runtime_minutes': 89, 'directors': ['Yuri Ilyenko'], 'writers': ['Yuri Ilyenko', 'Ivan Mikolaychuk'], 'producers': [], 'actors': ['Larisa Kadochnikova', 'Rodion Nakhapetov', 'Yuri Ilyenko', 'Nina Urgant', 'Valentina Vladimirova', 'Oleg Polstvin', 'Leonid Kuravlyov', 'Yuriy Mazhuga', 'Vladimir Alekseyenko', 'Ye. Andriyenko'], 'principals': [{'Leonid Grabovskiy': 'composer'}, {'Vilen Kalyuta': 'cinematographer'}, {'E. Sumovska': 'editor'}, {'Sergei Brzhestovsky': 'production_designer'}]}",
 "{'title': 'A 2000 pengös férfi', 'genres': ['Comedy', 'Romance'], 'is_adult': False, 'release_year': 1942, 'runtime_minutes': 88, 'directors': ['László Cserépy'], 'writers': ['Pál Barabás'], 'producers': [], 'actors': ['Elma Bulla', 'Erzsi Simor', 'Miklós Hajmássy', 'Piroska Vaszary', 'Ilona Kökény', 'Lajos Alszeghy', 'Róbert Bánky', 'Lajos Boray', 'Lajos Sugár', 'Márta Fónay'], 'principals': [{'Szabolcs Fény

In [8]:
document_prefix = "search_document: "

minicorpus_size = model.config.transductive_corpus_size
minicorpus_docs = random.sample(json_docs, minicorpus_size)
len(minicorpus_docs)

512

In [9]:
minicorpus_tokens = tokenizer(
    [document_prefix + doc for doc in minicorpus_docs],
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors="pt",
).to(device)

minicorpus_tokens["input_ids"].size()

torch.Size([512, 400])

In [17]:
dataloader = torch.utils.data.DataLoader(minicorpus_docs, batch_size=32, shuffle=False)

dataset_embeddings = []
for batch in tqdm(dataloader, smoothing=0):
    minicorpus_tokens = tokenizer(
        [document_prefix + doc for doc in batch],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt",
    ).to(device)
    with torch.no_grad():
        dataset_embeddings.append(model.first_stage_model(**minicorpus_tokens))

dataset_embeddings = torch.cat(dataset_embeddings)
dataset_embeddings.size()

  0%|          | 0/16 [00:00<?, ?it/s]

torch.Size([512, 768])

In [19]:
dataloader = torch.utils.data.DataLoader(json_docs, batch_size=32, shuffle=False)

doc_embeddings = []
for batch in tqdm(dataloader, smoothing=0):
    docs_batch = tokenizer(
        [document_prefix + doc for doc in batch],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt",
    ).to(device)
    with torch.no_grad():
        doc_embeddings_batch = model.second_stage_model(
            input_ids=docs_batch["input_ids"],
            attention_mask=docs_batch["attention_mask"],
            dataset_embeddings=dataset_embeddings,
        )
        doc_embeddings.append(doc_embeddings_batch)

doc_embeddings = torch.cat(doc_embeddings)
doc_embeddings /= doc_embeddings.norm(p=2, dim=1, keepdim=True)
doc_embeddings.size()

  0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([1600, 768])

In [20]:
df_subset_2 = df_subset.with_columns(embeds=doc_embeddings.cpu().numpy())
df_subset_2

tconst,startYear,averageRating,json,embeds
str,i64,f64,str,"array[f32, 768]"
"""tt0171517""",1975,6.2,"""{'title': 'Mechtat i zhit', 'g…","[-0.00642, -0.008363, … -0.095905]"
"""tt0034421""",1942,7.1,"""{'title': 'A 2000 pengös férfi…","[-0.027937, 0.038252, … -0.067883]"
"""tt0989642""",2006,5.6,"""{'title': 'Midnight Running', …","[-0.001267, 0.034308, … -0.027295]"
"""tt0273692""",1980,7.4,"""{'title': 'J.S. Brown, o Últim…","[-0.015184, 0.027515, … -0.077242]"
"""tt4470172""",2014,5.6,"""{'title': 'Guruldayan Kalpler'…","[-0.000246, 0.017736, … -0.053503]"
…,…,…,…,…
"""tt1326956""",2009,8.1,"""{'title': 'Our Summer in Tehra…","[-0.023254, 0.013627, … -0.036384]"
"""tt0175092""",1982,6.7,"""{'title': ""Puss 'n Boots"", 'ge…","[0.017054, 0.044542, … -0.076013]"
"""tt2739566""",1998,7.1,"""{'title': 'Jungle Love Story',…","[-0.049089, 0.011179, … -0.030443]"


In [21]:
df_subset_2.write_parquet(os.path.join(data_dir, "movie_embeds.parquet"))